## BM25 Ranking

In [1]:
# Load query and documents, inverted index and vocabulary

import json
import math
import time

with open("queries",'r') as file:
    query_dic = json.load(file)

with open("documents","r") as file:
    document_dic = json.load(file)
    
with open("Inverted_index","r") as file:
    Inverted_index = json.load(file)

with open("vocabulary","r") as file:
    vocabulary = file.readline()
    vocabulary = vocabulary.split(" ")

In [2]:
# Compute frequency for a query in document

def get_freq(query,document_id):
    
    doc = document_dic[document_id]  
    c = list(doc).count(query)
    freq = c/len(doc)
    
    return freq

In [3]:
#　ger average document lengths

def get_average(doc_dic):
    
    l = []
    for i in doc_dic.values():
        l.append(len(i))
    av = sum(l)/len(l)
    
    return av

Compute BM25 ranking score:

(Assume that r = R = 0)

In [4]:
R = 0
k1 = 1.2
b = 0.75


def bm25_score(n,f,r,N,dl,avdl):
    
    w_rsj = math.log( ( (r + 0.5) / (R - r + 0.5) ) / ( (n - r + 0.5) / (N - n - R + r + 0.5)) )
    B = (1-b) + b*dl/avdl
    r = (f / (k1* B + f)) * w_rsj
    
    return r

Get BM25 score:

In [5]:
def bm25(query):
    
    result = {}
    
    for token in query:
        if token in Inverted_index.keys():
            # Get posting list
            documents = Inverted_index[token]
            for doc in documents:
                freq = get_freq(token,doc)
                N =len(document_dic.keys())
                dl = len(document_dic[doc])
                avdl = get_average(document_dic)
                r = 0
                # get bm25 ranking score
                score = bm25_score(len(documents),freq,r,N,dl,avdl)
                # this document has been socred by other tokens
                if doc in result.keys():
                    result[doc] += score
                else:
                    result[doc] = score
                    
    return result
        

Rank scores:

In [6]:
# Ranking scores

def ranking(score):
    
    result = {}
    
    rank = sorted(score.items(),key=lambda x:x[1], reverse=True)
    
    # return top 100 results
    
    count = 0
    
    for i in rank:
        result[i[0]] = i[1]
        count += 1
        if count>100:
            break

    return result

Output to result file:
    
(Note: this step may take quite long time to run, in may machine: 7 mins)

In [ ]:
import time
t1 = time.time()

with open("output/result_bm25.txt","w") as file:
    for q in query_dic.items():
        query_id = q[0]
        query = q[1]
        scores = bm25(query)
        rank = ranking(scores)
        for r in rank.items():
            file.write(query_id)
            file.write(" ")
            file.write("1")
            file.write(" ")
            file.write(str(r[0]))
            file.write(" ")
            file.write("0")
            file.write(" ")
            file.write(str(r[1]))
            file.write(" ")
            file.write("run1")
            file.write('\n')
t2 = time.time()
print("running time :(/mins)", (t2-t1)/60)